In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix

MODEL_NAME = "LSOIE++"


def get_dev_result(model_name: str):
    model_path = "..\evaluations\\" + model_name + "_dev.txt"
    with open(model_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split(" ") for line in lines]
        y_true = [line[1] for line in lines if len(line) == 3]
        y_pred = [line[2] for line in lines if len(line) == 3]
        return y_true, y_pred
y_true, y_pred = get_dev_result(MODEL_NAME)

def get_confusion_matrix(y_true, y_pred):
    labels = list(set(y_true))
    df = pd.DataFrame(
        data=confusion_matrix(y_true, y_pred, labels=labels),
        columns=labels,
        index=labels,
    )
    df.index.name = "Y-True"
    df.columns.name = "Y-Pred"
    return df

#
# Local (metrics per class)
#
df = get_confusion_matrix(y_true, y_pred)

labels = list(set(y_true))
tps = {}
fps = {}
fns = {}
for label in labels:
    tps[label] = df.loc[label, label]
    fps[label] = df[label].sum() - tps[label]
    fns[label] = df.loc[label].sum() - tps[label]

#
# Global
#
micro_averages = {}
macro_averages = {}

correct_predictions = sum(tps.values())

total_predictions = df.values.sum()
accuracy_global = round(correct_predictions / total_predictions,4 ) if total_predictions > 0. else 0.

print("#-- Local measures --#")
print("True Positives:", tps)
print("False Positives:", fps)
print("False Negatives:", fns)

print("\n#-- Global measures --#")
print("Correct predictions:", correct_predictions)
print("Total predictions:", total_predictions)
print("Accuracy:", accuracy_global)

#-- Local measures --#
True Positives: {'B-ARG0': 28, 'B-ARG1': 11, 'I-ARG0': 35, 'I-V': 2, 'I-ARG1': 26, 'O': 499, 'B-V': 21}
False Positives: {'B-ARG0': 5, 'B-ARG1': 16, 'I-ARG0': 0, 'I-V': 0, 'I-ARG1': 28, 'O': 93, 'B-V': 5}
False Negatives: {'B-ARG0': 9, 'B-ARG1': 26, 'I-ARG0': 14, 'I-V': 38, 'I-ARG1': 23, 'O': 21, 'B-V': 16}

#-- Global measures --#
Correct predictions: 622
Total predictions: 769
Accuracy: 0.8088


In [2]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

      B-ARG0     0.8485    0.7568    0.8000        37
      B-ARG1     0.4074    0.2973    0.3438        37
         B-V     0.8077    0.5676    0.6667        37
      I-ARG0     1.0000    0.7143    0.8333        49
      I-ARG1     0.4815    0.5306    0.5049        49
         I-V     1.0000    0.0500    0.0952        40
           O     0.8429    0.9596    0.8975       520

    accuracy                         0.8088       769
   macro avg     0.7697    0.5537    0.5916       769
weighted avg     0.8157    0.8088    0.7842       769



In [3]:
df.style.background_gradient(cmap="Purples")

Y-Pred,B-ARG0,B-ARG1,I-ARG0,I-V,I-ARG1,O,B-V
Y-True,,,,,,,
B-ARG0,28,0,0,0,0,9,0
B-ARG1,0,11,0,0,8,18,0
I-ARG0,3,0,35,0,0,11,0
I-V,0,12,0,2,8,16,2
I-ARG1,0,0,0,0,26,23,0
O,2,4,0,0,12,499,3
B-V,0,0,0,0,0,16,21


In [27]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
sents = ["The world is very big", "I am a student", "I am a teacher", "I live in Brazil"]
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sent = "The Olympic Games take place every four years."
print(pipe(sent))


KeyboardInterrupt

